<a href="https://colab.research.google.com/github/samuelaojih/Google-Colab/blob/main/FloodSusceptibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
pip install earthengine-api

In [3]:
ee.Authenticate()
ee.Initialize(
    project ='ee-samuelojihcool',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [4]:
startyear = 2023
endyear = 2024
startDate = ee.Date.fromYMD(startyear, 1, 1)
endDate = ee.Date.fromYMD(endyear + 1, 1, 1)
years = list(range(startyear, endyear + 1))
geometry = ee.FeatureCollection('projects/ee-samuelojihcool/assets/cairo_basin')

# ---------------------- NORMALIZE FUNCTION ------------------------ #
def normalize(image, min_val, max_val):
    return image.subtract(min_val).divide(max_val - min_val)

# ------------------------ BASE DATASETS --------------------------- #
dem = ee.Image('USGS/SRTMGL1_003').clip(geometry)
slope = ee.Terrain.slope(dem)

clay = ee.Image("projects/soilgrids-isric/clay_mean").select('clay_0-5cm_mean').clip(geometry)
sand = ee.Image("projects/soilgrids-isric/sand_mean").select('sand_0-5cm_mean').clip(geometry)
silt = ee.Image("projects/soilgrids-isric/silt_mean").select('silt_0-5cm_mean').clip(geometry)

water = ee.ImageCollection('GLCF/GLS_WATER').filterBounds(geometry).max().clip(geometry)
waterMask = water.select('water').eq(2)
distWater = waterMask.fastDistanceTransform(100).sqrt().rename('distance')

flowaccum = ee.Image('WWF/HydroSHEDS/15ACC')
twi = flowaccum.add(1).multiply(ee.Image.pixelArea()) \
    .divide(slope.multiply(0.0174528).tan()).add(0.001).log().rename('TWI')

ndvi = ee.ImageCollection('MODIS/006/MOD13A1') \
    .filterDate(startDate, endDate).select('NDVI').mean().clip(geometry)

chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")
rain_images = [chirps.filter(ee.Filter.calendarRange(y, y, 'year')).sum().set('year', y) for y in years]
rainMean = ee.ImageCollection(rain_images).mean().clip(geometry)

# ------------------------- NORMALIZATION ---------------------------- #
normDEM = normalize(dem, -68, 1664)
normSlope = normalize(slope, 0, 78)
normClay = normalize(clay, 189, 468)
normSand = normalize(sand, 99, 684)
normSilt = normalize(silt, 107, 578)
normWaterDis = normalize(distWater, 0, 141.42)
normTWI = normalize(twi, -6.9, 24.28)
normNDVI = normalize(ndvi, -1978, 9282)
normRain = normalize(rainMean, 1102.67, 2178.03)

# ------------------------ CLASSIFICATION FUNCTION ------------------ #
def classify(image, weights):
    result = image
    result = result.where(image.lt(0.2), image.multiply(weights[0]))
    result = result.where(image.gte(0.2).And(image.lt(0.4)), image.multiply(weights[1]))
    result = result.where(image.gte(0.4).And(image.lt(0.6)), image.multiply(weights[2]))
    result = result.where(image.gte(0.6), image.multiply(weights[3]))
    return result

demClass = classify(normDEM, [3.50, 2.10, 0.70, 0.70])
slopeClass = classify(normSlope, [11.50, 6.90, 3.45, 1.15])
clayClass = classify(normClay, [1, 0.6, 0.4, 0.2])
sandClass = classify(normSand, [5, 4, 3, 2])
siltClass = classify(normSilt, [5, 4, 3, 2])
waterClass = classify(normWaterDis, [1.8, 0.6, 0.45, 0.15])
twiClass = classify(normTWI, [6.4, 4.8, 3.2, 1.6])
ndviClass = classify(normNDVI, [2.5, 1.5, 0.75, 0.25])
rainClass = classify(normRain, [3.3, 6.6, 9.9, 13.2])

# -------------------------- LULC ------------------------------- #
lulc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterDate(startDate, endDate).select('label').mosaic().clip(geometry)

lulcWeights = ee.Dictionary({
    0: 1.1, 1: 1.65, 2: 1.65, 3: 0.55, 4: 1.65,
    5: 2.2, 6: 1.1, 7: 0.55, 8: 0.55
})
remapVals = ee.List.sequence(0, 8)
weights = lulcWeights.values()
lulcWeighted = lulc.remap(remapVals, weights)

# --------------------- AHP Composite -------------------------- #
ahpRaw = demClass.add(slopeClass).add(clayClass).add(sandClass).add(siltClass) \
    .add(waterClass).add(twiClass).add(ndviClass).add(rainClass).add(lulcWeighted) \
    .rename("ahp")

ahpNorm = normalize(ahpRaw, 0.463, 27.593)

# ------------------------ VISUALIZATION ----------------------- #
Map = geemap.Map(center=[30.0444, 31.2357], zoom=8)

Map.addLayer(ahpNorm, {
    'min': 0,
    'max': 1,
    'palette': ['#3288bd', '#66c2a5', '#abdda4', '#e6f598', '#ffffbf',
                '#fee08b', '#fdae61', '#f46d43', '#d53e4f']
}, 'Flood Susceptibility (AHP)')

Map.addLayer(lulc, {
    'min': 0,
    'max': 8,
    'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6',
                '#E49635', '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1']
}, 'LULC Classes')

Map.addLayerControl()
Map

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A1

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[30.0444, 31.2357], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…